In [1]:
import os
import pickle
import time
import pandas as pd
import yfinance as yf
from fredapi import Fred
#from alpha_vantage.fundamentaldata import FundamentalData
from dotenv import load_dotenv
import pandas_ta as ta

load_dotenv()

# Set API keys
#ALPHA_VANTAGE_API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')
FRED_API_KEY = os.getenv('FRED_API_KEY')

#if not ALPHA_VANTAGE_API_KEY:
#    raise ValueError("Set Alpha Vantage API key in the '.env' file.")
if not FRED_API_KEY:
    raise ValueError("Set FRED API key in the '.env' file.")

fred = Fred(api_key=FRED_API_KEY)
#alpha_fd = FundamentalData(key=ALPHA_VANTAGE_API_KEY, output_format='pandas')


In [2]:
START_DATE = '2017-01-01'
END_DATE = '2023-12-31'


COMPANIES = {
    'Telus': ['T.TO'],
    'Rogers Communications': ['RCI-B.TO'],
    'Quebecor': ['QBR-B.TO'],
    'Cogeco Communications': ['CCA.TO'],
    'Bell Canada (BCE Inc.)': ['BCE.TO'],
    'Hydro One': ['H.TO'],
    'Fortis': ['FTS.TO'],
    'AltaGas': ['ALA.TO'],
    'Canadian National Railway': ['CNR.TO'],
    'Canadian Pacific Railway': ['CP.TO'],
    'Metro': ['MRU.TO'],
    'Loblaws': ['L.TO'],
    'Empire': ['EMP-A.TO'],
    'Alimentation Couche-Tard': ['ATD.TO'],
}


MACRO_INDICATORS = {
    'Real GDP': {'series_id': 'NGDPRSAXDCCAQ', 'frequency': 'Quarterly'},
    'GDP Growth Annual': {'series_id': 'CANGDPRQPSMEI', 'frequency': 'Annual'},
    'GDP Growth Quarterly': {'series_id': 'CANGDPRAPSMEI', 'frequency': 'Quarterly'},
    'CPI Growth Monthly': {'series_id': 'CPALTT01CAM659N', 'frequency': 'Monthly'},
    'CPI Growth Quarterly': {'series_id': 'CPALTT01CAQ657N', 'frequency': 'Quarterly'},
    'PPI Growth Monthly': {'series_id': 'CANPIEAMP01GPM', 'frequency': 'Monthly'},
    'PPI Growth Quarterly': {'series_id': 'CANPIEAMP01GPQ', 'frequency': 'Quarterly'},
    'Unemployment Rate': {'series_id': 'LRUNTTTTCAQ156S', 'frequency': 'Quarterly'},
    'Interest Rate': {'series_id': 'IRSTCB01CAM156N', 'frequency': 'Monthly'},
    'Industrial Production Ex Construction': {'series_id': 'PRINTO01CAQ189S', 'frequency': 'Quarterly'},
    'Industrial Production Growth Ex Construction': {'series_id': 'PRINTO01CAA657S', 'frequency': 'Annual'},
    'Industrial Production: Construction': {'series_id': 'PRINTO01CAQ189S', 'frequency': 'Quarterly'},
    'Industrial Production Growth: Construction': {'series_id': 'PRINTO01CAA657S', 'frequency': 'Annual'},
    'Consumer Confidence': {'series_id': 'CSCICP03CAM665S', 'frequency': 'Monthly'},
    'Retail Sales Growth': {'series_id': 'SLRTTO01CAA657S', 'frequency': 'Quarterly'},
    'Housing Starts': {'series_id': 'HSN1F', 'frequency': 'Monthly'},
    'Business Confidence': {'series_id': 'CANBSCICP02STSAQ', 'frequency': 'Quarterly'},
}

In [4]:
#def fetch_data_with_retry(fetch_function, *args, max_retries=3, sleep_time=15, **kwargs):
#    for attempt in range(max_retries):
#        try:
#            return fetch_function(*args, **kwargs)
#        except Exception as e:
#            print(f"Error: {e}. Retrying in {sleep_time} seconds...")
#            time.sleep(sleep_time)
#    print("Max retries exceeded.")
#    return None

In [3]:
def get_technical_data(companies, start_date, end_date):
    technical_data_list = []
    for company, tickers in companies.items():
        data_fetched = False
        for ticker in tickers:
            try:
                data = yf.download(ticker, start=start_date, end=end_date)
                if data.empty:
                    continue
                data.reset_index(inplace=True)
                data.columns = data.columns.get_level_values(0)
                # Calculate returns
                data['Daily Return'] = data['Close'].pct_change()
                data['Adjusted Daily Return'] = data['Adj Close'].pct_change()
                # Add company and ticker info
                data['Company'] = company
                data['Ticker'] = ticker
                # Calculate technical indicators
                data = calculate_technical_indicators(data)
                technical_data_list.append(data)
                print(f"Technical data fetched for {company} ({ticker})")
                data_fetched = True
                break
            except Exception as e:
                print(f"Error fetching technical data for {company} ({ticker}): {e}")
        if not data_fetched:
            print(f"Failed to fetch technical data for {company}")
    # Combine all data into one DataFrame
    technical_data = pd.concat(technical_data_list, ignore_index=True)
    return technical_data

def calculate_technical_indicators(data):
    # Moving Averages
    data['SMA_20'] = ta.sma(data['Adj Close'], length=20)
    data['SMA_100'] = ta.sma(data['Adj Close'], length=100)
    data['EMA_20'] = ta.ema(data['Adj Close'], length=20)
    data['EMA_100'] = ta.ema(data['Adj Close'], length=100)
    # Bollinger Bands
    bbands = ta.bbands(data['Adj Close'], length=20)
    data = pd.concat([data, bbands], axis=1)
    # Relative Strength Index
    data['RSI_14'] = ta.rsi(data['Adj Close'], length=14)
    # Moving Average Convergence Divergence
    macd = ta.macd(data['Adj Close'])
    data = pd.concat([data, macd], axis=1)
    return data

def melt_technical_data(data):
    """
    Melt the technical data DataFrame into long format.
    """
    id_vars = ['Company', 'Ticker', 'Date']
    value_vars = [col for col in data.columns if col not in id_vars]
    melted_data = data.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='Variable',
        value_name='Value'
    )
    return melted_data

In [4]:
technical_data = get_technical_data(COMPANIES, START_DATE, END_DATE)
technical_data['Date'] = pd.to_datetime(technical_data['Date']).dt.date
technical_data = technical_data[technical_data['Date'] >= pd.to_datetime('2018-01-01').date()]
technical_melted = melt_technical_data(technical_data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Technical data fetched for Telus (T.TO)



[*********************100%***********************]  1 of 1 completed

Technical data fetched for Rogers Communications (RCI-B.TO)



[*********************100%***********************]  1 of 1 completed


...

Technical data fetched for Canadian Pacific Railway (CP.TO)
Technical data fetched for Metro (MRU.TO)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Technical data fetched for Loblaws (L.TO)
Technical data fetched for Empire (EMP-A.TO)


[*********************100%***********************]  1 of 1 completed


Technical data fetched for Alimentation Couche-Tard (ATD.TO)


In [7]:
def get_macro_data(indicators, start_date, end_date):
    macro_data_list = []
    for indicator_name, info in indicators.items():
        series_id = info['series_id']
        frequency = info['frequency']
        try:
            data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
            data = data.to_frame(name='Value')
            data['Indicator'] = indicator_name
            data['Frequency'] = frequency
            data.index.rename('Date', inplace=True)
            data.reset_index(inplace=True)
            macro_data_list.append(data)
            
            print(f"Macro data fetched for {indicator_name}")
        except Exception as e:
            print(f"Error fetching macro data for {indicator_name} ({series_id}): {e}")

    # Combine all data into one DataFrame
    macro_data = pd.concat(macro_data_list, ignore_index=True)
    return macro_data

In [8]:
macro_data = get_macro_data(MACRO_INDICATORS, START_DATE, END_DATE)

Macro data fetched for Real GDP
Macro data fetched for GDP Growth Annual
Macro data fetched for GDP Growth Quarterly
Macro data fetched for CPI Growth Monthly
Macro data fetched for CPI Growth Quarterly
Macro data fetched for PPI Growth Monthly
Macro data fetched for PPI Growth Quarterly
Macro data fetched for Unemployment Rate
Macro data fetched for Interest Rate
Macro data fetched for Industrial Production Ex Construction
Macro data fetched for Industrial Production Growth Ex Construction
Macro data fetched for Industrial Production: Construction
Macro data fetched for Industrial Production Growth: Construction
Macro data fetched for Consumer Confidence
Macro data fetched for Retail Sales Growth
Macro data fetched for Housing Starts
Macro data fetched for Business Confidence


In [5]:
def get_recent_info(companies):
   tickers_data= {}
   for company, tickers in companies.items():
      for ticker in tickers:
         ticker_object = yf.Ticker(ticker)
         #convert info() output from dictionary to dataframe
         temp = pd.DataFrame.from_dict(ticker_object.info, orient="index")
         temp.reset_index(inplace=True)
         temp['Company'] = company
         temp.columns = ["Attribute", "Recent", "Company"]
         
         # add (ticker, dataframe) to main dictionary
         tickers_data[ticker] = temp

   combined_data = pd.concat(tickers_data)
   combined_data = combined_data.reset_index()
   del combined_data["level_1"] 
   combined_data.columns = ["Ticker", "Attribute", "Recent", "Company"] 
   return combined_data

In [ ]:
actual_data = get_recent_info(COMPANIES)

In [ ]:
# Save the outputs as pickle files
#with open('macro_data.pkl', 'wb') as macro_file:
#    pickle.dump(macro_data, macro_file)

#with open('technical_data.pkl', 'wb') as technical_file:
#    pickle.dump(technical_data, technical_file)

#with open('technical_melted.pkl', 'wb') as melted_file:
#    pickle.dump(technical_melted, melted_file)

Plots

In [ ]:
import pandas as pd
import plotly.graph_objects as go


ticker_df = technical_melted[technical_melted['Ticker'] == 'T.TO']

indicators = ["SMA_20", "EMA_20", "EMA_100", "SMA_100", "RSI_14", "MACD_12_26_9"]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ticker_df[ticker_df["Variable"] == "Adj Close"]["Date"],
    y=ticker_df[ticker_df["Variable"] == "Adj Close"]["Value"],
    mode='lines',
    name='Adj close',
    line=dict(width=2)
))

for indicator in indicators:
    fig.add_trace(go.Scatter(
        x=ticker_df[ticker_df["Variable"] == indicator]["Date"],
        y=ticker_df[ticker_df["Variable"] == indicator]["Value"],
        mode='lines',
        name=indicator,
        visible=(indicator == "SMA_20")  # Make SMA_20 visible by default
    ))

fig.add_trace(go.Bar(
    x=ticker_df[ticker_df["Variable"] == "Volume"]["Date"],
    y=ticker_df[ticker_df["Variable"] == "Volume"]["Value"],
    name='Volume',
    opacity=0.5,
    yaxis='y2'
))

buttons = [
    {
        "label": indicator,
        "method": "update",
        "args": [{"visible": [True] + [ind == indicator for ind in indicators] + [True]}]
    } for indicator in indicators
]

fig.update_layout(
    title='Stock Analysis',
    xaxis_title='Date',
    yaxis=dict(title='Price', side='left'),
    yaxis2=dict(
        title='Volume',
        overlaying='y',
        side='right',
        showgrid=False,
        range=[0, ticker_df[ticker_df['Variable'] == 'Volume']['Value'].max() * 1.2]
    ),
    legend_title='Variables',
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "showactive": True,
        "x": 0.1,
        "y": 1.15
    }]
)


fig.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go


ticker_df = technical_melted[technical_melted['Ticker'] == 'T.TO']


ticker_df_adj_close = ticker_df[ticker_df['Variable'] == 'Adj Close'].copy()  # Use .copy() to avoid SettingWithCopyWarning
ticker_df_adj_close.loc[:, 'Volatility'] = ticker_df_adj_close['Value'].pct_change().rolling(window=20).std() * (252 ** 0.5)


fig_volatility = go.Figure(data=[go.Scatter(
    x=ticker_df_adj_close['Date'],
    y=ticker_df_adj_close['Volatility'],
    mode='lines',
    name='Volatility',
    line=dict(width=2)
)])


fig_volatility.update_layout(
    title='Volatility Chart (20-day Rolling)',
    xaxis_title='Date',
    yaxis_title='Volatility',
    template='plotly_white'
)


fig_volatility.show()



Sectorial Info

In [ ]:
# Defining the sector mapping
sector_mapping = {
    'ATD.TO': 'consumer_staples',
    'EMP-A.TO': 'consumer_staples',
    'L.TO': 'consumer_staples',
    'MRU.TO': 'consumer_staples',
    'CNR.TO': 'industry_rail',
    'CP.TO': 'industry_rail',
    'ALA.TO': 'utilities',
    'FTS.TO': 'utilities',
    'H.TO': 'utilities',
    'BCE.TO': 'telecom',
    'CCA.TO': 'telecom',
    'QBR-B.TO': 'telecom',
    'RCI-B.TO': 'telecom',
    'T.TO': 'telecom'
}

# Adding the 'Sector' column based on the mapping
actual_data['Sector'] = actual_data['Ticker'].map(sector_mapping)

relevant_attributes = [
    'dividendRate', 'dividendYield', 'beta', 'forwardPE', 'marketCap',
    'profitMargins', 'shortRatio', 'priceToBook', 'freeCashflow', 'ebitdaMargins'
]

filtered_data = actual_data[actual_data['Attribute'].isin(relevant_attributes)]
def display_stock_info(selected_ticker):
    sector = actual_data.loc[actual_data['Ticker'] == selected_ticker, 'Sector'].values[0]
    sector_data = filtered_data[filtered_data['Sector'] == sector]
    sector_table = sector_data.pivot(index=['Company', 'Ticker'], columns='Attribute', values='Recent')
    print(f"\nStock and Peers Information for {selected_ticker}:")
    return sector_table

display_stock_info('H.TO')




Stock and Peers Information for H.TO:


,Attribute,beta,dividendRate,dividendYield,ebitdaMargins,forwardPE,freeCashflow,marketCap,priceToBook,profitMargins,shortRatio
Company,Ticker,,,,,,,,,,
AltaGas,ALA.TO,1.23,1.19,0.0353,0.12559,14.785088,61375000,10038567936,1.315307,0.04112,2.99
Fortis,FTS.TO,0.229,2.46,0.0412,0.44979,17.886229,-382124992,29583249408,1.400309,0.14352,6.86
Hydro One,H.TO,0.342,1.26,0.0281,0.34069,22.138615,-954625024,26806734848,2.251649,0.13847,2.54


Interative plot

In [21]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Filter for Adj Close data and create a copy
ticker_df = technical_melted[technical_melted['Ticker'] == 'T.TO']
ticker_df_adj_close = ticker_df[ticker_df['Variable'] == 'Adj Close'].copy()

# Calculate daily returns and common metrics
ticker_df_adj_close['Daily Return'] = ticker_df_adj_close['Value'].pct_change()
ticker_df_adj_close['Volatility_20d'] = ticker_df_adj_close['Daily Return'].rolling(20).std() * (252 ** 0.5)
ticker_df_adj_close['Volatility_60d'] = ticker_df_adj_close['Daily Return'].rolling(60).std() * (252 ** 0.5)
ticker_df_adj_close['EWMA Volatility'] = ticker_df_adj_close['Daily Return'].ewm(span=20).std() * (252 ** 0.5)

# Maximum drawdown
ticker_df_adj_close['Cumulative Return'] = (1 + ticker_df_adj_close['Daily Return']).cumprod()
ticker_df_adj_close['Max Drawdown'] = (ticker_df_adj_close['Cumulative Return'] / ticker_df_adj_close['Cumulative Return'].cummax() - 1).cummin()

# Expected shortfall (5% level)
def expected_shortfall(series):
    series = series.dropna()
    return series[series <= series.quantile(0.05)].mean()

ticker_df_adj_close['Expected Shortfall'] = ticker_df_adj_close['Daily Return'].rolling(20).apply(expected_shortfall, raw=False)

# Create plot with traces
fig_volatility = go.Figure()

metrics = {
    '20-Day Volatility': 'Volatility_20d',
    '60-Day Volatility': 'Volatility_60d',
    'EWMA Volatility': 'EWMA Volatility',
    'Max Drawdown': 'Max Drawdown',
    'Expected Shortfall': 'Expected Shortfall',
    'Cumulative Return': 'Cumulative Return'
}

for name, col in metrics.items():
    fig_volatility.add_trace(go.Scatter(
        x=ticker_df_adj_close['Date'],
        y=ticker_df_adj_close[col],
        mode='lines',
        name=name,
        visible=(name == '20-Day Volatility')
    ))

# Create buttons for each metric
buttons = [
    dict(label=name,
         method="update",
         args=[{"visible": [name == key for key in metrics.keys()]},
               {"title": name}])
    for name in metrics.keys()
]

# Update layout with buttons
fig_volatility.update_layout(
    updatemenus=[{
        "buttons": buttons,
        "direction": "down",
        "showactive": True,
        "x": 0.1,
        "y": 1.15
    }],
    title='20-Day Volatility',
    xaxis_title='Date',
    yaxis_title='Metric Value',
    template='plotly_white'
)

fig_volatility.show()

